# Database in SQL

Kaggle competition https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting

In [1]:
import pylab as py
import pandas as pandas
from datetime import datetime
import seaborn as sns
import sqlite3
import os

In [2]:
DATABASE_FILENAME = 'restaurant_forecasting.sqlite'
POPULATE_DATABASE = False
def get_database():
    database = sqlite3.connect(DATABASE_FILENAME)
    return database
def reset_database():
    os.remove(DATABASE_FILENAME)
    return get_database()

## Creating database

In [3]:
if POPULATE_DATABASE:
    database = reset_database()
else:
    database = get_database()

In [4]:
if POPULATE_DATABASE:
    table = 'reserves'
    columns = {
        'service': 'CHAR(3)',
        'air_store_id': 'CHAR(20)',
        'hpg_store_id': 'CHAR(20)',
        'visit_datetime': 'DATETIME',
        'reserve_datetime': 'DATETIME',
        'reserve_visitors': 'UNSIGNED SMALLINT',
    }
    sql_command = 'CREATE TABLE {name:} ( {column_names:} ) ;'.format(**{
        'name': table,
        'column_names': ' , '.join([' {} {} '.format(x,y) for x,y in columns.items()])
    })
    try:
        database.execute(sql_command)
        database.commit()
    except sqlite3.OperationalError:
        pass

In [5]:
if POPULATE_DATABASE:
    table = 'stores'
    columns = {
        'hpg_store_id': 'CHAR(20) UNIQUE',
        'hpg_genre_name': 'TINYTEXT',
        'hpg_area_name': 'TINYTEXT',
        'hpg_latitude': 'FLOAT',
        'hpg_longitude': 'FLOAT',
        'air_store_id': 'CHAR(20) UNIQUE',
        'air_genre_name': 'TINYTEXT',
        'air_area_name': 'TINYTEXT',
        'air_latitude': 'FLOAT',
        'air_longitude': 'FLOAT',
    }
    sql_command = 'CREATE TABLE {name:} ( {column_names:} ) ;'.format(**{
        'name': table,
        'column_names': ' , '.join([' {} {} '.format(x,y) for x,y in columns.items()])
    })
    try:
        database.execute(sql_command)
        database.commit()
    except sqlite3.OperationalError:
        pass

In [6]:
if POPULATE_DATABASE:
    table = 'visits'
    columns = {
        'air_store_id': 'CHAR(20)',
        'hpg_store_id': 'CHAR(20)',
        'visit_date': 'DATE',
        'visitors': 'UNSIGNED SMALLINT',
    }
    sql_command = 'CREATE TABLE {name:} ( {column_names:} ) ;'.format(**{
        'name': table,
        'column_names': ' , '.join([' {} {} '.format(x,y) for x,y in columns.items()])
    })
    try:
        database.execute(sql_command)
        database.commit()
    except sqlite3.OperationalError:
        pass

## Insert reservations made with HPG

In [7]:
if POPULATE_DATABASE:
    data_hpg_reserve = pandas.read_csv('Data/hpg_reserve.csv')

In [8]:
if POPULATE_DATABASE:
    for k,reserve in data_hpg_reserve.iterrows():
        add = {
            'service': 'HPG',
            'hpg_store_id': reserve['hpg_store_id'],
            'visit_datetime': reserve['visit_datetime'],
            'reserve_datetime': reserve['reserve_datetime'],
            'reserve_visitors': reserve['reserve_visitors'],
        }
        sql_command = 'INSERT INTO reserves ( {columns:} ) VALUES ( {values:} )'.format(**{
            'columns': ' , '.join(add.keys()),
            'values': ' , '.join([':'+x for x in add.keys()]),
        })
        database.execute(sql_command, add)
    database.commit()

## Insert reservations made with AIR

In [9]:
if POPULATE_DATABASE:
    data_air_reserve = pandas.read_csv('Data/air_reserve.csv')

In [10]:
if POPULATE_DATABASE:
    for k,reserve in data_air_reserve.iterrows():
        add = {
            'service': 'AIR',
            'air_store_id': reserve['air_store_id'],
            'visit_datetime': reserve['visit_datetime'],
            'reserve_datetime': reserve['reserve_datetime'],
            'reserve_visitors': reserve['reserve_visitors'],
        }
        sql_command = 'INSERT INTO reserves ( {columns:} ) VALUES ( {values:} )'.format(**{
            'columns': ' , '.join(add.keys()),
            'values': ' , '.join([':'+x for x in add.keys()]),
        })
        database.execute(sql_command, add)
    database.commit()

## Insert visit data obtained from AIR

In [11]:
if POPULATE_DATABASE:
    data_air_visit = pandas.read_csv('Data/air_visit_data.csv')

In [12]:
if POPULATE_DATABASE:
    for k,reserve in data_air_visit.iterrows():
        add = {
            'air_store_id': reserve['air_store_id'],
            'visit_date': reserve['visit_date'],
            'visitors': reserve['visitors'],
        }
        sql_command = 'INSERT INTO visits ( {columns:} ) VALUES ( {values:} )'.format(**{
            'columns': ' , '.join(add.keys()),
            'values': ' , '.join([':'+x for x in add.keys()]),
        })
        database.execute(sql_command, add)
    database.commit()

## Ling AIR id with HPG id

In [13]:
if POPULATE_DATABASE:
    relation = pandas.read_csv('Data/store_id_relation.csv')

In [14]:
if POPULATE_DATABASE:
    sql_commands = []
    sql_commands.append( """
        UPDATE reserves
        SET air_store_id = :air_store_id
        WHERE hpg_store_id = :hpg_store_id ;
        """ )
    sql_commands.append( """
        UPDATE reserves
        SET hpg_store_id = :hpg_store_id
        WHERE air_store_id = :air_store_id ;
        """ )
    sql_commands.append( """
        UPDATE visits
        SET hpg_store_id = :hpg_store_id
        WHERE air_store_id = :air_store_id ;
        """ )
    for k,link in relation.iterrows():
        dic = {
            'air_store_id': link['air_store_id'],
            'hpg_store_id': link['hpg_store_id'],
        }
        for sql_command in sql_commands:
            database.execute(sql_command,dic)
    database.commit()

## Store information

In [15]:
if POPULATE_DATABASE:
    relation = pandas.read_csv('Data/store_id_relation.csv')
    hpg_store_info = pandas.read_csv('Data/hpg_store_info.csv')
    air_store_info = pandas.read_csv('Data/air_store_info.csv')
    relation_ids = {x:y for x,y in zip(relation['air_store_id'],relation['hpg_store_id'])}    

In [16]:
if POPULATE_DATABASE:
    for k, store in hpg_store_info.iterrows():
        add = {
            'hpg_store_id': store['hpg_store_id'],
            'hpg_genre_name': store['hpg_genre_name'],
            'hpg_area_name': store['hpg_area_name'],
            'hpg_latitude': store['latitude'],
            'hpg_longitude': store['longitude'],
        }
        sql_command = 'INSERT INTO stores ( {columns:} ) VALUES ( {values:} ) ;'.format(**{
            'columns': ' , '.join(add.keys()),
            'values': ' , '.join([':' + x for x in add.keys()]),
        })
        try:
            database.execute(sql_command, add)
        except sqlite3.IntegrityError:
            pass
            
    for k, store in air_store_info.iterrows():
        add = {
            'air_store_id': store['air_store_id'],
            'air_genre_name': store['air_genre_name'],
            'air_area_name': store['air_area_name'],
            'air_latitude': round(store['latitude'],6),
            'air_longitude': round(store['longitude'],6),
        }
        if store['air_store_id'] in relation_ids.keys():
            sql_command = 'UPDATE stores SET {set_clause:} WHERE hpg_store_id = :hpg_store_id ;'.format(**{
                'set_clause': ' , '.join(['{} = :{}'.format(x,x) for x in add.keys()])
            })
            hpg_id = relation_ids[store['air_store_id']]
            add.update({'hpg_store_id': hpg_id})
            try:
                database.execute(sql_command, add)
            except sqlite3.IntegrityError:
                pass

        else:
            sql_command = 'INSERT INTO stores ( {columns:} ) VALUES ( {values:} ) ;'.format(**{
                'columns': ' , '.join(add.keys()),
                'values': ' , '.join([':' + x for x in add.keys()]),
            })
            try:
                database.execute(sql_command, add)
            except sqlite3.IntegrityError:
                pass

database.commit()

## How many common restaurants, and how popular are they?

In [17]:
sql_command = """
    SELECT * FROM stores
    WHERE hpg_store_id IS NOT NULL
    AND air_store_id IS NOT NULL
    """
stores_common = database.execute(sql_command).fetchall()
sql_command = """
    SELECT * FROM stores
    WHERE hpg_store_id IS NOT NULL
    AND air_store_id IS NULL
    """
stores_onlyHPG = database.execute(sql_command).fetchall()
sql_command = """
    SELECT * FROM stores
    WHERE hpg_store_id IS NULL
    AND air_store_id IS NOT NULL
    """
stores_onlyAIR = database.execute(sql_command).fetchall()
sql_command = """
    SELECT * FROM stores
    WHERE hpg_store_id IS NULL
    AND air_store_id IS NULL
    """
stores_neither = database.execute(sql_command).fetchall()

In [18]:
print(len(stores_common))
print(len(stores_onlyHPG))
print(len(stores_onlyAIR))
print(len(stores_neither))

63
4627
679
0


## Are all restaurants represented?

In [19]:
columns = {
    'reserves.service' : 'service',
    'reserves.hpg_store_id' : 'hpg_store_id',
    'reserves.air_store_id' : 'air_store_id',
    'reserves.visit_datetime' : 'visit_datetime',
    'reserves.reserve_datetime' : 'reserve_datetime',
    'reserves.reserve_visitors' : 'reserve_visitors',
    'stores.hpg_genre_name' : 'hpg_genre_name',
    'stores.hpg_area_name' : 'hpg_area_name',
    'stores.air_genre_name' : 'air_genre_name',
    'stores.air_area_name' : 'air_area_name',
    'stores.hpg_latitude' : 'hpg_latitude',
    'stores.hpg_longitude' : 'hpg_longitude',
    'stores.air_latitude' : 'air_latitude',
    'stores.air_longitude' : 'air_longitude',
}

In [20]:
cursor = database.cursor()
sql_command = """
    SELECT {}
    FROM reserves LEFT JOIN stores 
    ON (reserves.hpg_store_id = stores.hpg_store_id OR reserves.air_store_id = stores.air_store_id)
    ORDER BY reserves.visit_datetime
    ; """.format(','.join(columns.keys()),','.join(columns.keys()))
execution = cursor.execute(sql_command)
results_all = cursor.fetchall()
reserves_all = pandas.DataFrame(results_all)
reserves_all.columns = columns.values()

In [21]:
reserves_inHPG = [x for x in results_all if x[0]=='HPG']
reserves_inAIR = [x for x in results_all if x[0]=='AIR']

In [22]:
print(len(reserves_inHPG) / len(results_all))
print(len(reserves_inAIR) / len(results_all))

0.9558569846198544
0.04414301538014563


In [23]:
reserves_common = [x for x in results_all if x[8]!=None and x[6]!=None]
reserves_onlyAIR = [x for x in results_all if x[8]!=None and x[6]==None]
reserves_onlyHPG = [x for x in results_all if x[8]==None and x[6]!=None]
reserves_neither = [x for x in results_all if x[8]==None and x[6]==None]

In [24]:
print(len(reserves_common) / len(results_all))
print(len(reserves_onlyHPG) / len(results_all))
print(len(reserves_onlyAIR) / len(results_all))
print(len(reserves_neither) / len(results_all))

0.014183604132082125
0.4098919194265011
0.0268141891472157
0.5491102872942011


## Summary of store information about reservations

There are a total of about 2M (2092698) reservations.

With respect to the service used, the breakdown is
* 95.6% were done in the HPG system.
* 4.4% were done in the AIR system.

With respect to stores, the breakdown is
* 1.4% were done in 63 restaurants present in both AIR and HPG (of these, 54.4% were done with AIR and 45.6% were done with HPG).
* 2.7% were done in 183 restaurants present only in the AIR system.
* 41.0% were done in 4627 restaurants present only in the HPG system.
* 54.9% were done in 8722 restaurants for which no information is available (of these, 1.8% were done with AIR and 98.2% were done with HPG).


## Export data

In [25]:
with open('reservations.csv','w') as f:
    reserves_all.to_csv(f,index=False)

# Visitation data

In [26]:
columns = {
    'visits.hpg_store_id' : 'hpg_store_id',
    'visits.air_store_id' : 'air_store_id',
    'visits.visit_date' : 'visit_date',
    'visits.visitors' : 'visitors',
    'stores.hpg_genre_name' : 'hpg_genre_name',
    'stores.hpg_area_name' : 'hpg_area_name',
    'stores.air_genre_name' : 'air_genre_name',
    'stores.air_area_name' : 'air_area_name',
    'stores.hpg_latitude' : 'hpg_latitude',
    'stores.hpg_longitude' : 'hpg_longitude',
    'stores.air_latitude' : 'air_latitude',
    'stores.air_longitude' : 'air_longitude',
}

In [27]:
cursor = database.cursor()
sql_command = """
    SELECT {}
    FROM visits LEFT JOIN stores
    ON (visits.air_store_id = stores.air_store_id or visits.hpg_store_id = stores.hpg_store_id)
    ORDER BY visits.visit_date
    ; """.format(','.join(columns.keys()),','.join(columns.keys()))
execution = cursor.execute(sql_command)
results_all = cursor.fetchall()
visits_all = pandas.DataFrame(results_all)
visits_all.columns = columns.values()

In [28]:
with open('visits.csv','w') as f:
    visits_all.to_csv(f,index=False)